In [1]:
import pandas as pd
from IPython.display import display
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az
import matplotlib.pyplot as plt
import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

In [2]:
marketing_spend= pd.read_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/advertising_raw.csv')

/var/folders/kk/x9w58sq1639gdzg86gw_hfg00000gp/T/ipykernel_97270/1764015688.py:1: DtypeWarning: Columns (25,32) have mixed types. Specify dtype option on import or set low_memory=False.
  marketing_spend= pd.read_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/advertising_raw.csv')


In [3]:
def process_marketing_spend(df):
    """
    Process marketing spend dataframe to create platform-specific spend columns
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with marketing_platform_name and marketing_spend columns
    
    Returns:
    pandas.DataFrame: Processed dataframe with platform-specific spend columns
    """
    # List of platforms
    platforms = ['reddit', 'linkedin', 'facebook', 'google', 'bing', 
                'tiktok', 'twitter', 'instagram']
    
    # Create new spend columns initialized to 0
    for platform in platforms:
        df[f'{platform}_spend'] = 0
        
    # Iterate through each row and assign spend to appropriate column
    for idx, row in df.iterrows():
        platform = row['marketing_platform_name'].lower()
        if platform in platforms:
            df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']
    
    return df

def process_marketing_impressions(df):
    """
    Process marketing spend dataframe to create platform-specific spend columns
    
    Parameters:
    df (pandas.DataFrame): Input dataframe with marketing_platform_name and marketing_spend columns
    
    Returns:
    pandas.DataFrame: Processed dataframe with platform-specific spend columns
    """
    # List of platforms
    platforms = ['reddit', 'linkedin', 'facebook', 'google', 'bing', 
                'tiktok', 'twitter', 'instagram']
    
    # Create new spend columns initialized to 0
    for platform in platforms:
        df[f'{platform}_impressions'] = 0
        
    # Iterate through each row and assign spend to appropriate column
    for idx, row in df.iterrows():
        platform = row['marketing_platform_name'].lower()
        if platform in platforms:
            df.at[idx, f'{platform}_spend'] = row['impressions']
    
    return df

# Example usage:
# marketing_spend_processed = process_marketing_spend(marketing_spend)

# To verify the results:
# print(marketing_spend_processed.head())


In [4]:
processed_marketing_spend = process_marketing_spend(marketing_spend)
processed_marketing_spend = process_marketing_impressions(processed_marketing_spend)

/var/folders/kk/x9w58sq1639gdzg86gw_hfg00000gp/T/ipykernel_97270/623930320.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '147.37319' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']
/var/folders/kk/x9w58sq1639gdzg86gw_hfg00000gp/T/ipykernel_97270/623930320.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2231.85' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, f'{platform}_spend'] = row['campaign_spend_eur']
/var/folders/kk/x9w58sq1639gdzg86gw_hfg00000gp/T/ipykernel_97270/623930320.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.02' has dtype incompatible with int64, please e

In [5]:
df_web_summit = processed_marketing_spend[processed_marketing_spend['event_name'].str.contains('Web Summit 2023')]
#df_web_summit.to_csv('./ws_attempt/data/raw_data/marketing_spend_processed_new_approach_web_summit.csv')

In [6]:
numeric_columns = df_web_summit.select_dtypes(include=['int64', 'float64']).columns
df_web_summit_grouped = df_web_summit.groupby('date_id')[numeric_columns].sum().reset_index()
df_web_summit_grouped= df_web_summit_grouped.interpolate(method='linear')
df_web_summit_grouped.to_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/marketing_spend_processed_new_approach_web_summit_grouped.csv')

/var/folders/kk/x9w58sq1639gdzg86gw_hfg00000gp/T/ipykernel_97270/3103639258.py:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_web_summit_grouped= df_web_summit_grouped.interpolate(method='linear')


In [15]:

# Add this code to your notebook

# 1. First, let's check the data types
print("Data types before conversion:")
print(df_web_summit[['date_id', 'event_date']].dtypes)

# 2. Convert date columns to datetime
df_web_summit['date_id'] = pd.to_datetime(df_web_summit['date_id'])
df_web_summit['event_date'] = pd.to_datetime(df_web_summit['event_date'])

# 3. Verify conversion
print("\nData types after conversion:")
print(df_web_summit[['date_id', 'event_date']].dtypes)

# 4. Check for any rows where conversion failed (would be NaT)
print("\nRows with invalid dates:")
print(df_web_summit[df_web_summit['date_id'].isna()].shape[0])
print(df_web_summit[df_web_summit['event_date'].isna()].shape[0])

# 5. Identify your geo column (using reach_country_id)
geo_column = 'reach_country_id'

# 6. Check unique values in geo column
unique_geos = df_web_summit[geo_column].unique()
print(f"\nNumber of unique geos: {len(unique_geos)}")
print(f"Sample geos: {unique_geos[:5]}")

# 7. Find min and max dates
min_date = df_web_summit['date_id'].min()
max_date = df_web_summit['date_id'].max()
print(f"\nDate range: {min_date} to {max_date}")

# 8. Sort by geo and date
df_web_summit = df_web_summit.sort_values([geo_column, 'date_id'])

# 9. Calculate days between dates within each geo group
# Use a safer approach that handles the first row in each group
df_web_summit['days_diff'] = df_web_summit.groupby(geo_column)['date_id'].diff().dt.days

# 10. Check the distribution of day differences
days_diff_counts = df_web_summit['days_diff'].value_counts().head(10)
print("\nMost common day differences:")
print(days_diff_counts)

# 11. Choose a frequency based on the most common interval
# For marketing data, weekly is often appropriate
freq = 'W'  # Weekly

# 12. Create a complete date range
date_range = pd.date_range(start=min_date, end=max_date, freq=freq)
print(f"\nCreated {len(date_range)} dates with frequency {freq}")

# 13. Create a MultiIndex for all geo-date combinations
# Filter out any problematic geos (like '---' or empty strings)
valid_geos = [geo for geo in unique_geos if geo not in ['---', '', None]]
if len(valid_geos) == 0:
    # If no valid geos, create a dummy one
    valid_geos = ['default_geo']
    df_web_summit[geo_column] = 'default_geo'

complete_index = pd.MultiIndex.from_product(
    [valid_geos, date_range],
    names=[geo_column, 'date_id']
)

# 14. Prepare numeric columns for aggregation
numeric_cols = df_web_summit.select_dtypes(include=['number']).columns.tolist()
# Remove any columns you don't want to aggregate
if 'Unnamed: 0' in numeric_cols:
    numeric_cols.remove('Unnamed: 0')

# 15. Aggregate data by geo and date
print("\nAggregating data by geo and date...")
aggregated_df = df_web_summit.groupby([geo_column, 'date_id'])[numeric_cols].sum().reset_index()

# 16. Reindex to include all geo-date combinations
print("Reindexing to include all geo-date combinations...")
complete_df = aggregated_df.set_index([geo_column, 'date_id']).reindex(complete_index)

# 17. Fill missing values with interpolation within each geo
print("Filling missing values with interpolation...")
for geo in valid_geos:
    geo_mask = complete_df.index.get_level_values(geo_column) == geo
    complete_df.loc[geo_mask] = complete_df.loc[geo_mask].interpolate(method='linear')

# 18. Fill any remaining NaNs with 0
complete_df = complete_df.fillna(0)

# 19. Reset index to get geo and date back as columns
regularized_df = complete_df.reset_index()

# This is removing the geos and also summing the resulting data
#regularized_df['reach_country_id'] = "TOTAl"
regularized_df['geo'] = "TOTAL"
regularized_df = regularized_df.groupby('date_id').sum(numeric_only=True).reset_index()



# 20. Save to CSV
output_path = '/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/regularized_web_summit_data.csv'
regularized_df.to_csv(output_path, index=False)

print(f"\nData has been regularized and saved to '{output_path}'")
print(f"Shape of original data: {df_web_summit.shape}")
print(f"Shape of regularized data: {regularized_df.shape}")


Data types before conversion:
date_id       datetime64[ns]
event_date    datetime64[ns]
dtype: object

Data types after conversion:
date_id       datetime64[ns]
event_date    datetime64[ns]
dtype: object

Rows with invalid dates:
0
0

Number of unique geos: 240
Sample geos: ['---' 'ABW' 'AFG' 'AGO' 'AIA']

Date range: 2022-10-14 00:00:00 to 2024-06-30 00:00:00

Most common day differences:
days_diff
0.0     121744
1.0      55034
2.0       2213
3.0       1038
4.0        604
5.0        285
6.0        187
10.0       157
7.0        145
14.0       140
Name: count, dtype: int64

Created 90 dates with frequency W

Aggregating data by geo and date...
Reindexing to include all geo-date combinations...
Filling missing values with interpolation...

Data has been regularized and saved to '/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/regularized_web_summit_data.csv'
Shape of original data: (182693, 53)
Shape of regularized data: (90, 29)


In [8]:
regularized_df['date_id'].value_counts()

date_id
2022-10-16    1
2024-01-28    1
2024-01-14    1
2024-01-07    1
2023-12-31    1
2023-12-24    1
2023-12-17    1
2023-12-10    1
2023-12-03    1
2023-11-26    1
2023-11-19    1
2023-11-12    1
2023-11-05    1
2023-10-29    1
2023-10-22    1
2023-10-15    1
2023-10-08    1
2023-10-01    1
2023-09-24    1
2023-09-17    1
2023-09-10    1
2024-01-21    1
2024-02-04    1
2022-10-23    1
2024-02-11    1
2024-06-23    1
2024-06-16    1
2024-06-09    1
2024-06-02    1
2024-05-26    1
2024-05-19    1
2024-05-12    1
2024-05-05    1
2024-04-28    1
2024-04-21    1
2024-04-14    1
2024-04-07    1
2024-03-31    1
2024-03-24    1
2024-03-17    1
2024-03-10    1
2024-03-03    1
2024-02-25    1
2024-02-18    1
2023-09-03    1
2023-08-27    1
2023-08-20    1
2023-08-13    1
2023-03-05    1
2023-02-26    1
2023-02-19    1
2023-02-12    1
2023-02-05    1
2023-01-29    1
2023-01-22    1
2023-01-15    1
2023-01-08    1
2023-01-01    1
2022-12-25    1
2022-12-18    1
2022-12-11    1
2022-12-04    1


In [9]:
regularized_df[regularized_df['date_id'] == '2022-10-16'].head()

,date_id,conference_year,event_currency_id,currency_id,impressions,marketing_platform_id,clicks,reach,campaign_spend_eur,days_to_event,campaign_spend,weeks_to_event,reddit_spend,linkedin_spend,facebook_spend,google_spend,bing_spend,tiktok_spend,twitter_spend,instagram_spend,reddit_impressions,linkedin_impressions,facebook_impressions,google_impressions,bing_impressions,tiktok_impressions,twitter_impressions,instagram_impressions,days_diff
0,2022-10-16,946764.0,468.0,468.0,3018606.0,693.0,85845.0,385186.0,5238.031181,183924.0,5238.031181,26208.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,232.0


In [17]:
coord_to_columns = load.CoordToColumns(
    time='date_id',
    geo='geo',
    controls=[],
    #population='population',
    kpi='clicks',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'linkedin_impressions',
        'google_impressions',
        'bing_impressions',
        'tiktok_impressions',
        'twitter_impressions',
        'reddit_impressions',
        'instagram_impressions',
    ],
    media_spend=[
        'facebook_spend',
        'linkedin_spend',
        'google_spend',
        'bing_spend',
        'tiktok_spend',
        'twitter_spend',
        'reddit_spend',
        'instagram_spend',
    ],
    #organic_media=['Organic_channel0_impression'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'linkedin_impressions': 'LinkedIn',
    'google_impressions': 'Google',
    'bing_impressions': 'Bing',
    'tiktok_impressions': 'TickTock',
    'twitter_impressions': 'Twitter',
    'reddit_impressions': 'Reddit',
    'instagram_impressions': 'Instagram',
}
correct_media_spend_to_channel = {
    'facebook_spend': 'Facebook',
    'linkedin_spend': 'LinkedIn',
    'google_spend': 'Google',
    'bing_spend': 'Bing',
    'tiktok_spend': 'TickTock',
    'twitter_spend': 'Twitter',
    'reddit_spend': 'Reddit',
    'instagram_spend': 'Instagram',
}



loader = load.CsvDataLoader(
    csv_path="/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/regularized_web_summit_data.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)


data = loader.load()

roi_mu = 1 #0.2     # Mu for ROI prior for each media channel.
roi_sigma = 5 #0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior, media_effects_dist='normal')

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=2000, seed=1)

ValueError: geos already set to ['national_geo'].

In [21]:
# Normalize your data before feeding it to Meridian
# For each numeric column that varies widely
def normalise_column(df):
    columns = [
        'impressions',  'clicks',
       'campaign_spend_eur', 'campaign_spend',
        'reddit_spend', 'linkedin_spend', 'facebook_spend',
       'google_spend', 'bing_spend', 'tiktok_spend', 'twitter_spend',
       'instagram_spend', 'reddit_impressions', 'linkedin_impressions',
       'facebook_impressions', 'google_impressions', 'bing_impressions',
       'tiktok_impressions', 'twitter_impressions', 'instagram_impressions',
       ]
    for col in columns:
        if col in df.columns:
            df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df


In [22]:
regularized_df = pd.DataFrame(regularized_df)
regularized_df.to_csv('/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/regularized_normalised_df_2.csv', index=False)

In [26]:
coord_to_columns = load.CoordToColumns(
    time='date_id',
    #geo='reach_country_id',
    controls=[],
    #population='population',
    kpi='clicks',
    #revenue_per_kpi='revenue_per_conversion',
    media=[
        'facebook_impressions',
        'linkedin_impressions',
        'google_impressions',
        'bing_impressions',
        'tiktok_impressions',
        'twitter_impressions',
        'reddit_impressions',
        'instagram_impressions',
    ],
    media_spend=[
        'facebook_spend',
        'linkedin_spend',
        'google_spend',
        'bing_spend',
        'tiktok_spend',
        'twitter_spend',
        'reddit_spend',
        'instagram_spend',
    ],
    #organic_media=['Organic_channel0_impression'],
    #non_media_treatments=['Promo'],
)

correct_media_to_channel = {
    'facebook_impressions': 'Facebook',
    'linkedin_impressions': 'LinkedIn',
    'google_impressions': 'Google',
    'bing_impressions': 'Bing',
    'tiktok_impressions': 'TickTock',
    'twitter_impressions': 'Twitter',
    'reddit_impressions': 'Reddit',
    'instagram_impressions': 'Instagram',
}
correct_media_spend_to_channel = {
    'facebook_spend': 'Facebook',
    'linkedin_spend': 'LinkedIn',
    'google_spend': 'Google',
    'bing_spend': 'Bing',
    'tiktok_spend': 'TickTock',
    'twitter_spend': 'Twitter',
    'reddit_spend': 'Reddit',
    'instagram_spend': 'Instagram',
}



loader = load.CsvDataLoader(
    csv_path="/Users/aaronmeagher/Work/google_meridian/google/ws_attempt/data/raw_data/regularized_normalised_df_2.csv",
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)


data = loader.load()

roi_mu = 1 #0.2     # Mu for ROI prior for each media channel.
roi_sigma = 5 #0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior, media_effects_dist='normal')

mmm = model.Meridian(input_data=data, model_spec=model_spec)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=2000, seed=1)

ValueError: Duplicate entries found in the 'time' column.

In [25]:
print(mmm.geos)

NameError: name 'mmm' is not defined